In [203]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [205]:
import os
import pandas as pd

# Relative path from notebook/ to processed/
data_path = os.path.join("..", "data", "processed", "Ocean_Pearl_Recommendation.csv")

print("Looking for:", data_path)

raw_data = pd.read_csv(data_path)


Looking for: ..\data\processed\Ocean_Pearl_Recommendation.csv


In [207]:
raw_data

,Recipe ID,Dish Name,Category,Type,Cuisine,Rating,Prep Time (mins),Nutrients,Taste Profile & Tags,Ingredients,...,Review,Spice Level,Serving Size,Allergens,Availability,Chef's Special,Calories,Pairing Suggestions,Taste Profile,Tags
0,R1,Tomato Soup,Starter,Veg,Continental,4.5,20,120 kcal / 4g protein / 20g carbs / 3g fat,"Savory, tangy; Tags: Vegetarian, Healthy, Quick","Tomatoes, cream, garlic, onion, butter, salt, ...",...,"Rich flavor profile, always satisfying.",Mild,Large,Dairy,Always,No,120,"Garlic Bread, Soup","Savory, Tangy","Healthy, Quick, Vegetarian"
1,R2,Mushroom Soup,Starter,Veg,Continental,4.4,25,150 kcal / 6g protein / 18g carbs / 4g fat,"Earthy, savory; Tags: Vegetarian, Healthy","Mushrooms, cream, butter, garlic, herbs",...,"Rich flavor profile, always satisfying.",Mild,Medium,Dairy,Seasonal,No,150,"French Fries, Coke",Savory,"Healthy, Vegetarian"
2,R3,Chicken Soup,Starter,Non-Veg,Continental,4.6,30,160 kcal / 15g protein / 12g carbs / 5g fat,"Light, savory; Tags: Non-Veg, Healthy","Chicken, stock, garlic, onion, spices",...,"Comforting and tasty, great with sides.",Mild,Large,NaN,Seasonal,No,160,"Plain Rice, Green Salad",Savory,"Healthy, Non-veg"
3,R4,Chicken Biryani,Rice Meal,Non-Veg,Indian,4.8,50,520 kcal / 25g protein / 70g carbs / 18g fat,"Spicy, aromatic; Tags: Non-Veg, Traditional","Rice, chicken, saffron, spices, onions",...,Authentic taste that feels homemade.,Medium,Medium,NaN,Seasonal,Yes,520,"Raita, Salad",Spicy,Non-veg
4,R5,Mutton Biryani,Rice Meal,Non-Veg,Indian,4.7,60,600 kcal / 28g protein / 72g carbs / 22g fat,"Rich, spicy; Tags: Non-Veg, Traditional","Rice, mutton, saffron, spices, onions",...,"Rich flavor profile, always satisfying.",Medium,Small,NaN,Limited,No,600,"Raita, Salad",Spicy,Non-veg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,R129,Rajma Chawal,Main Course,Veg,Indian (North),4.8,50,420 kcal / 20g protein / 60g carbs / 10g fat,"Comforting, spicy; Tags: Veg, Classic","Kidney beans, rice, onion, tomato, spices",...,Absolutely delicious and worth trying again!,Mild,Large,Dairy,Limited,No,420,"Tandoori Roti, Dal Fry",Spicy,General
129,R130,Fish Curry (Goan),Main Course,Non-Veg,Indian (Goan),4.8,48,460 kcal / 28g protein / 24g carbs / 24g fat,"Spicy, tangy; Tags: Non-Veg, Coastal","Fish, coconut milk, tamarind, spices",...,Loved by both kids and adults alike.,Spicy,Medium,Seafood,Limited,Yes,460,"Steamed Rice, Rasam","Spicy, Tangy",Non-veg
130,R131,Butter Naan,Breads,Veg,Indian,4.7,18,280 kcal / 8g protein / 48g carbs / 6g fat,"Soft, buttery; Tags: Veg, Side","Flour, butter, yogurt, yeast",...,"Great value for the price, must-try dish.",Medium,Medium,Dairy,Limited,No,280,"Garlic Bread, Soup",Neutral,General
131,R132,Garlic Naan,Breads,Veg,Indian,4.8,20,300 kcal / 8g protein / 50g carbs / 8g fat,"Soft, garlicky; Tags: Veg, Side","Flour, garlic, butter, yeast",...,"Light, fresh, and very enjoyable.",Mild,Medium,Dairy,Limited,Yes,300,"Tandoori Roti, Dal Fry",Neutral,General


## Select imp Features
1. Recipe ID
2. Dish Name
3. Category
4. Type
5. Cuisine
6. Rating
7. Ingredients
8. Price (₹)
9. 'Review'
10. Spice Level
11. Taste Profile
12. Tags

In [210]:
raw_data.columns

Index(['Recipe ID', 'Dish Name', 'Category', 'Type', 'Cuisine', 'Rating',
       'Prep Time (mins)', 'Nutrients', 'Taste Profile & Tags', 'Ingredients',
       'Price (₹)', 'Order Count', 'Time of Day', 'Review', 'Spice Level',
       'Serving Size', 'Allergens', 'Availability', 'Chef's Special',
       'Calories', 'Pairing Suggestions', 'Taste Profile', 'Tags'],
      dtype='object')

In [212]:
data = raw_data[['Recipe ID', 'Dish Name', 'Category', 'Type', 'Cuisine', 'Rating',
        'Ingredients','Price (₹)', 'Review', 'Spice Level','Taste Profile', 'Tags']]

In [214]:
data.head()

,Recipe ID,Dish Name,Category,Type,Cuisine,Rating,Ingredients,Price (₹),Review,Spice Level,Taste Profile,Tags
0,R1,Tomato Soup,Starter,Veg,Continental,4.5,"Tomatoes, cream, garlic, onion, butter, salt, ...",180,"Rich flavor profile, always satisfying.",Mild,"Savory, Tangy","Healthy, Quick, Vegetarian"
1,R2,Mushroom Soup,Starter,Veg,Continental,4.4,"Mushrooms, cream, butter, garlic, herbs",200,"Rich flavor profile, always satisfying.",Mild,Savory,"Healthy, Vegetarian"
2,R3,Chicken Soup,Starter,Non-Veg,Continental,4.6,"Chicken, stock, garlic, onion, spices",220,"Comforting and tasty, great with sides.",Mild,Savory,"Healthy, Non-veg"
3,R4,Chicken Biryani,Rice Meal,Non-Veg,Indian,4.8,"Rice, chicken, saffron, spices, onions",440,Authentic taste that feels homemade.,Medium,Spicy,Non-veg
4,R5,Mutton Biryani,Rice Meal,Non-Veg,Indian,4.7,"Rice, mutton, saffron, spices, onions",490,"Rich flavor profile, always satisfying.",Medium,Spicy,Non-veg


# Text Cleaning

In [217]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shaik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shaik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [219]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [221]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()                          # lowercase
    text = re.sub(r'[^a-z\s]', '', text)         # remove punctuation & numbers
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

In [223]:
# Apply cleaning
data["Ingredients"] = data["Ingredients"].apply(clean_text)
data["Review"] = data["Review"].apply(clean_text)
data["Tags"] = data["Tags"].apply(clean_text)

# One-Hot Encode Categorical Features

In [226]:
categorical_cols = ["Type", "Cuisine", "Category", "Spice Level"]

In [228]:
for col in categorical_cols:
    dummies = pd.get_dummies(data[col], prefix=col, drop_first=False)
    dummies = dummies.astype(bool)  # convert to True/False
    data.drop(columns=[col], inplace=True)
    data = pd.concat([data, dummies], axis=1)

#  Normalize Numeric Features

In [231]:
scaler = MinMaxScaler()
numeric_cols = ["Rating", "Price (₹)"]

In [233]:
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

#  Text Vectorization (TF-IDF)

In [236]:
tfidf_ingredients = TfidfVectorizer(max_features=500)
tfidf_tags = TfidfVectorizer(max_features=300)
tfidf_reviews = TfidfVectorizer(max_features=500)


In [238]:
ingredients_tfidf = tfidf_ingredients.fit_transform(data["Ingredients"].fillna(""))
tags_tfidf = tfidf_tags.fit_transform(data["Tags"].fillna(""))
reviews_tfidf = tfidf_reviews.fit_transform(data["Review"].fillna(""))

In [240]:
ingredients_df = pd.DataFrame(ingredients_tfidf.toarray(), columns=tfidf_ingredients.get_feature_names_out())
tags_df = pd.DataFrame(tags_tfidf.toarray(), columns=tfidf_tags.get_feature_names_out())
reviews_df = pd.DataFrame(reviews_tfidf.toarray(), columns=tfidf_reviews.get_feature_names_out())


#  Merge Final Features

In [243]:
data = pd.concat([data.reset_index(drop=True), ingredients_df, tags_df, reviews_df], axis=1)

In [245]:
cleaned_data = data.copy()

# Compute Cosine Similarity

In [248]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

# Step 1: Drop non-feature columns
feature_cols = cleaned_data.drop(columns=["Recipe ID", "Dish Name", "Ingredients", "Review", "Tags"])

# Step 2: Separate numeric and categorical columns
numeric_cols = feature_cols.select_dtypes(include=['int64', 'float64'])
categorical_cols = feature_cols.select_dtypes(include=['object'])

# Step 3: One-hot encode categorical features (like Taste, Cuisine, etc.)
if not categorical_cols.empty:
    encoder = OneHotEncoder()
    encoded_cats = encoder.fit_transform(categorical_cols).toarray()
else:
    encoded_cats = None

# Step 4: Combine numeric + encoded categorical
import numpy as np
if encoded_cats is not None:
    final_features = np.hstack([numeric_cols.values, encoded_cats])
else:
    final_features = numeric_cols.values

# Step 5: Compute cosine similarity
cosine_sim = cosine_similarity(final_features)

print("Cosine similarity shape:", cosine_sim.shape)

Cosine similarity shape: (133, 133)


# Recommendation Function

In [251]:
def recommend_dishes(dish_name, top_n=5):
    if dish_name not in cleaned_data["Dish Name"].values:
        return f"❌ '{dish_name}' not found in dataset."
    
    # Get index of the dish
    idx = cleaned_data.index[cleaned_data["Dish Name"] == dish_name].tolist()[0]
    
    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort by similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Pick top N (excluding itself)
    sim_scores = sim_scores[1:top_n+1]
    
    # Get recommended dish IDs
    dish_indices = [i[0] for i in sim_scores]
    return cleaned_data.iloc[dish_indices][["Recipe ID", "Dish Name"]].to_dict(orient="records")

#  Test the System

In [254]:
recs = recommend_dishes("Paneer Butter Masala", top_n=5)
print("Recommended Dishes:\n", recs)

Recommended Dishes:
 [{'Recipe ID': 'R91', 'Dish Name': 'Paneer Butter Masala'}, {'Recipe ID': 'R51', 'Dish Name': 'Shahi Paneer'}, {'Recipe ID': 'R105', 'Dish Name': 'Malai Kofta'}, {'Recipe ID': 'R129', 'Dish Name': 'Rajma Chawal'}, {'Recipe ID': 'R126', 'Dish Name': 'Paneer Butter Masala'}]
